BigMart Sales Prediction practice problem
We have train (8523) , train data set has both input and output variable(s). We need to predict the sales for test data set.(Regression) and predict the type of fat of each product (classification) and grouping the products similarity based on product visibility and product maximum retail price (clustering). 

Item_Identifier: Unique product ID

Item_Weight: Weight of product

Item_Fat_Content: Whether the product is low fat or not
    
Item_Visibility: The % of total display area of all products in a store allocated to the particular product

Item_Type: The category to which the product belongs

Item_MRP: Maximum Retail Price (list price) of the product

Outlet_Identifier: Unique store ID

Outlet_Establishment_Year: The year in which store was established

Outlet_Size: The size of the store in terms of ground area covered

Outlet_Location_Type: The type of city in which the store is located

Outlet_Type: Whether the outlet is just a grocery store or some sort of supermarket

Item_Outlet_Sales: Sales of the product in the particulat store. This is the outcome variable to be predicted

# Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go
from xgboost import XGBRegressor
import plotly.express as px

# Data collection and analysis

In [2]:
big_mart_data = pd.read_csv("/kaggle/input/big-mart-sales/Train.csv")
# big_mart_data=pd.concat([big_mart_data,pd.read_csv("D:\\04.1Ultrasonic_Sensor\\archive (9) 2\\Test_modified.csv")[:1504]])
# big_mart_data=pd.concat([big_mart_data,pd.read_csv("D:\\04.1Ultrasonic_Sensor\\archive (9) 2\\Test.csv")[:1504]])

In [3]:
# first 5 rows of the dataframe
big_mart_data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
duplicate_rows = big_mart_data[big_mart_data.duplicated(keep=False)]

len(duplicate_rows)


0

In [5]:
big_mart_data.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [6]:
# number of data points & number of features
big_mart_data.shape

(8523, 12)

In [7]:
# getting some information about thye dataset
big_mart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [8]:
# checking for missing values
big_mart_data.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [9]:
#Box Plot suggest we dont have any outlier and hence we can change missing values with 'Mean'
fig = px.box(big_mart_data, y='Item_Weight', title='Item Weight Distribution using box plot (no outliers) ')
fig.update_layout(width=800, height=500)  
fig.show()

In [10]:
# mean value of "Item_Weight" column
big_mart_data['Item_Weight'].mean()

12.857645184135976

In [11]:
# filling the missing values in "Item_weight column" with "Mean" value
big_mart_data['Item_Weight'].fillna(big_mart_data['Item_Weight'].mean(), inplace=True)

/tmp/ipykernel_18/2509980927.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [12]:
big_mart_data.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [13]:

big_mart_data['Outlet_Size'].mode()

0    Medium
Name: Outlet_Size, dtype: object

In [14]:
# Count occurrences of each combination
counts = big_mart_data.groupby(['Outlet_Size', 'Outlet_Type']).size().unstack(fill_value=0)
print(counts)
# Plot using Plotly
fig = px.bar(counts, barmode='stack')

# Customize the layout
fig.update_layout(
    title='Outlet Type vs Outlet Size',
    xaxis_title='Outlet Type',
    yaxis_title='Count',
    xaxis={'categoryorder': 'total descending'},  # Sort categories by total count
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    margin=dict(l=50, r=50, t=50, b=50)  # Add margin for better spacing
)

# Show the plot
fig.show()


Outlet_Type  Grocery Store  Supermarket Type1  Supermarket Type2  \
Outlet_Size                                                        
High                     0                932                  0   
Medium                   0                930                928   
Small                  528               1860                  0   

Outlet_Type  Supermarket Type3  
Outlet_Size                     
High                         0  
Medium                     935  
Small                        0  


/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [15]:
mode_of_outlet_size = big_mart_data.pivot_table(values='Outlet_Size',columns='Outlet_Type',aggfunc=lambda x: x.mode()[0])

In [16]:
mode_of_outlet_size

Outlet_Type,Grocery Store,Supermarket Type1,Supermarket Type2,Supermarket Type3
Outlet_Size,Small,Small,Medium,Medium


In [17]:
miss_values = big_mart_data['Outlet_Size'].isnull()  
miss_values

0       False
1       False
2       False
3        True
4       False
        ...  
8518    False
8519     True
8520    False
8521    False
8522    False
Name: Outlet_Size, Length: 8523, dtype: bool

In [18]:
# filling the missing values in "Outlet_Size" column with Mode
#Here we take Outlet_Size column & Outlet_Type column since they are correlated
big_mart_data.loc[miss_values, 'Outlet_Size'] = big_mart_data.loc[miss_values,'Outlet_Type'].apply(lambda x: mode_of_outlet_size[x])
big_mart = big_mart_data

In [19]:
# checking for missing values
big_mart_data.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

# Data Visualization

In [20]:
# Create a histogram trace
histogram_trace = go.Histogram(x=big_mart_data['Item_Weight'], 
                               marker=dict(color='blue', opacity=0.5))

# Create layout
layout = go.Layout(title='Item Weight Distribution (after missing values handling)',
                   xaxis=dict(title='Item Weight'),
                   yaxis=dict(title='Frequency'))

# Create figure
fig = go.Figure(data=[histogram_trace], layout=layout)

# Show the plot
fig.show()


In [21]:
#Box Plot suggest we dont have any outlier and hence we can change missing values with 'Mean'
fig = px.box(big_mart_data, y='Item_Weight', title='Item Weight Distribution')
fig.update_layout(width=800, height=500)  
fig.show()

In [22]:
# Create a histogram trace
histogram_trace = go.Histogram(x=big_mart_data['Item_Visibility'], 
                               marker=dict(color='green', opacity=0.5))

# Create layout
layout = go.Layout(title='Item Visibility Distribution (before analysis)',
                   xaxis=dict(title='Item Visibility'),
                   yaxis=dict(title='Frequency')) 

# Create figure
fig = go.Figure(data=[histogram_trace], layout=layout)

# Show the plot
fig.show()



In [23]:
big_mart_data.loc[big_mart_data['Item_Visibility'] == 0, 'Item_Visibility'] = big_mart_data['Item_Visibility'].mean()

In [24]:
big_mart_data['Item_Visibility'] = np.log(1+big_mart_data['Item_Visibility'])
# Create a histogram trace
histogram_trace = go.Histogram(x=big_mart_data['Item_Visibility'], 
                               marker=dict(color='green', opacity=0.5))
# Create layout
layout = go.Layout(title='Item Visibility Distribution (after analysis)',
                   xaxis=dict(title='Item Visibility'),
                   yaxis=dict(title='Frequency')) 
# Create figure
fig = go.Figure(data=[histogram_trace], layout=layout)

# Show the plot
fig.show()

In [25]:
# Plotting with Plotly
fig = go.Figure()

# Adding histogram
fig.add_trace(go.Histogram(x=big_mart_data['Item_MRP'], name='Histogram'))



# Layout settings
fig.update_layout(title='Distribution of Item_MRP',
                  xaxis_title='Item_MRP',
                  yaxis_title='Frequency')

fig.show()

In [26]:
# Plotting with Plotly
fig = go.Figure()

# Adding histogram
fig.add_trace(go.Histogram(x=big_mart_data['Item_Outlet_Sales'], name='Histogram'))



# Layout settings
fig.update_layout(title='Distribution of Item_Outlet_Sales (before analysis)',
                  xaxis_title='Item_Outlet_Sales',
                  yaxis_title='Frequency')

fig.show()

In [27]:
big_mart_data.drop(big_mart_data[big_mart_data['Item_Outlet_Sales'] <= 0].index, inplace=True)
big_mart_data['Item_Outlet_Sales'].fillna(big_mart_data['Item_Outlet_Sales'].mean(), inplace=True)
big_mart_data['Item_Outlet_Sales'] = np.log(1+big_mart_data['Item_Outlet_Sales'])
#big_mart_data['Item_Outlet_Sales'] = np.exp(big_mart_data['Item_Outlet_Sales']) - 1 while reverse to orgiinal

/tmp/ipykernel_18/2315809334.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [28]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=big_mart_data['Item_Outlet_Sales'], name='Histogram'))
fig.update_layout(title='Distribution of Item_Outlet_Sales (after analysis)',
                  xaxis_title='Item_Outlet_Sales',
                  yaxis_title='Frequency')

fig.show()

In [29]:
big_mart_data.shape

(8523, 12)

In [30]:
import plotly.express as px

fig = px.histogram(big_mart_data, x='Outlet_Establishment_Year',color_discrete_sequence=px.colors.qualitative.Pastel, title='Outlet Establishment Year Distribution')

fig.update_layout(xaxis_title='Outlet Establishment Year', yaxis_title='Count', 
                  xaxis=dict(type='category'), barmode='group')
fig.update_xaxes(categoryorder='category ascending')

fig.show()

In [31]:
import plotly.express as px

# Create a count plot using Plotly Express
fig = px.histogram(big_mart_data, x='Item_Fat_Content')

# Update layout
fig.update_layout(xaxis_title='Item_Fat_Content', yaxis_title='Count', 
                  xaxis=dict(type='category'), barmode='group')

# Show the plot
fig.show()

In [32]:
import plotly.express as px

# Create a count plot using Plotly Express
fig = px.histogram(big_mart_data, x='Item_Type')

# Update layout
fig.update_layout(xaxis_title='Item_Type', yaxis_title='Count', 
                  xaxis=dict(type='category'), barmode='group')

# Show the plot
fig.show()

In [33]:


# Create the scatter plot using Plotly
fig = go.Figure(data=go.Scatter(x=big_mart_data['Item_Outlet_Sales'],
                                y=big_mart_data['Item_Visibility'],
                                mode='markers'))

# Update layout
fig.update_layout(title='Item Visibility vs. Item Outlet Sales',
                  xaxis_title='Item Outlet Sales',
                  yaxis_title='Item Visibility')

# Show the plot
fig.show()


In [34]:


# Create the scatter plot using Plotly
fig = go.Figure(data=go.Scatter(x=big_mart_data['Item_Outlet_Sales'],
                                y=big_mart_data['Item_MRP'],
                                mode='markers'))

# Update layout
fig.update_layout(title='Item MRP vs. Item Outlet Sales',
                  xaxis_title='Item Outlet Sales',
                  yaxis_title='Item MRP')

# Show the plot
fig.show()

In [35]:
# Plot the bubble plot
fig = px.scatter(big_mart_data, x='Item_Outlet_Sales', y='Item_MRP', size='Item_Weight', color='Item_Fat_Content',
                 hover_name='Item_Identifier', log_x=True, log_y=True,
                 title='Bubble Plot of Item MRP vs Outlet Sales',
                 labels={'Item_MRP': 'Item MRP', 'Item_Outlet_Sales': 'Outlet Sales'})

fig.show()

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



# Label Encoding

In [36]:
big_mart_data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.015920,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,8.225808
1,DRC01,5.92,Regular,0.019095,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,6.096776
2,FDN15,17.50,Low Fat,0.016621,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,7.648868
3,FDX07,19.20,Regular,0.064037,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,6.597664
4,NCD19,8.93,Low Fat,0.064037,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,6.903451


In [37]:
big_mart_data['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: count, dtype: int64

In [38]:
big_mart_data.replace({'Item_Fat_Content': {'low fat':'Low Fat','LF':'Low Fat', 'reg':'Regular'}}, inplace=True)

In [39]:
big_mart_data['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    5517
Regular    3006
Name: count, dtype: int64

In [40]:
encoder = LabelEncoder()

In [41]:
big_mart_data['Item_Identifier'] = encoder.fit_transform(big_mart_data['Item_Identifier'])

big_mart_data['Item_Fat_Content'] = encoder.fit_transform(big_mart_data['Item_Fat_Content'])

big_mart_data['Item_Type'] = encoder.fit_transform(big_mart_data['Item_Type'])

big_mart_data['Outlet_Identifier'] = encoder.fit_transform(big_mart_data['Outlet_Identifier'])

big_mart_data['Outlet_Size'] = encoder.fit_transform(big_mart_data['Outlet_Size'])

big_mart_data['Outlet_Location_Type'] = encoder.fit_transform(big_mart_data['Outlet_Location_Type'])

big_mart_data['Outlet_Type'] = encoder.fit_transform(big_mart_data['Outlet_Type'])

In [42]:
big_mart_data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,156,9.30,0,0.015920,4,249.8092,9,1999,1,0,1,8.225808
1,8,5.92,1,0.019095,14,48.2692,3,2009,1,2,2,6.096776
2,662,17.50,0,0.016621,10,141.6180,9,1999,1,0,1,7.648868
3,1121,19.20,1,0.064037,6,182.0950,0,1998,2,2,0,6.597664
4,1297,8.93,0,0.064037,9,53.8614,1,1987,0,2,1,6.903451


Splitting features and Target

In [43]:
X = big_mart_data.drop(columns='Item_Outlet_Sales', axis=1)
Y = big_mart_data['Item_Outlet_Sales']

In [44]:
X

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,156,9.300,0,0.015920,4,249.8092,9,1999,1,0,1
1,8,5.920,1,0.019095,14,48.2692,3,2009,1,2,2
2,662,17.500,0,0.016621,10,141.6180,9,1999,1,0,1
3,1121,19.200,1,0.064037,6,182.0950,0,1998,2,2,0
4,1297,8.930,0,0.064037,9,53.8614,1,1987,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...
8518,370,6.865,0,0.055230,13,214.5218,1,1987,0,2,1
8519,897,8.380,1,0.045912,0,108.1570,7,2002,2,1,1
8520,1357,10.600,0,0.034581,8,85.1224,6,2004,2,1,1
8521,681,7.210,1,0.135597,13,103.1332,3,2009,1,2,2


In [45]:
Y

0       8.225808
1       6.096776
2       7.648868
3       6.597664
4       6.903451
          ...   
8518    7.929984
8519    6.310436
8520    7.085159
8521    7.521100
8522    6.642056
Name: Item_Outlet_Sales, Length: 8523, dtype: float64

In [46]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=2)

In [47]:
print(X.shape, X_train.shape, X_test.shape)

(8523, 11) (5966, 11) (2557, 11)


# Machine Learning Model for regression

# XGBoost Regressor

In [48]:
regressor = XGBRegressor()

In [49]:
regressor.fit(X_train, Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [50]:
pd.DataFrame({
    'feature':X.columns,
    'XGBRF_importance':regressor.fit(X_train, Y_train).feature_importances_
    
}).sort_values(by='XGBRF_importance',ascending=False)

,feature,XGBRF_importance
10,Outlet_Type,0.926887
5,Item_MRP,0.030763
7,Outlet_Establishment_Year,0.011626
9,Outlet_Location_Type,0.005021
2,Item_Fat_Content,0.004160
1,Item_Weight,0.003932
3,Item_Visibility,0.003730
8,Outlet_Size,0.003663
4,Item_Type,0.003603
6,Outlet_Identifier,0.003349


# Evaluation

In [51]:
# prediction on training data
training_data_prediction = regressor.predict(X_train)

In [52]:
# R squared Value
r2_train = metrics.r2_score(Y_train, training_data_prediction)

In [53]:
print('R Squared value = ', r2_train)

R Squared value =  0.9122228652863242


In [54]:
# prediction on test data
test_data_prediction = regressor.predict(X_test)

In [55]:
from sklearn.model_selection import cross_val_score
# R squared Value
r2_test = metrics.r2_score(Y_test, test_data_prediction)

# predict the training set
pred = regressor.predict(X)

# perform cross-validation
cv_score = cross_val_score(regressor, X, Y, scoring='r2', cv=5)
cv_score = np.abs(np.mean(cv_score))

print("Model Report")
print("CV Score:", cv_score)

Model Report
CV Score: 0.6845027640882337


In [56]:
print('R Squared value = ', r2_test)

R Squared value =  0.7008335597040496


In [57]:
mse = mean_squared_error(Y_test, test_data_prediction)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.31830210392789726


# Ridge

In [58]:
from sklearn.linear_model import Ridge
regressor = Ridge(alpha=0.1)  # alpha is the regularization parameter
regressor.fit(X_train, Y_train)

Ridge(alpha=0.1)

# Evaluation

In [59]:
# prediction on training data
training_data_prediction = regressor.predict(X_train)

In [60]:
# R squared Value
r2_train = metrics.r2_score(Y_train, training_data_prediction)

In [61]:
print('R Squared value = ', r2_train)

R Squared value =  0.5744924232148398


In [62]:
# prediction on test data
test_data_prediction = regressor.predict(X_test)

In [63]:
# R squared Value
r2_test = metrics.r2_score(Y_test, test_data_prediction)

# predict the training set
pred = regressor.predict(X)

# perform cross-validation
cv_score = cross_val_score(regressor, X, Y, scoring='r2', cv=5)
cv_score = np.abs(np.mean(cv_score))

print("Model Report")
print("CV Score:", cv_score)

Model Report
CV Score: 0.5780129259634664


In [64]:
print('R Squared value = ', r2_test)

R Squared value =  0.5915008937950537


In [65]:
mse = mean_squared_error(Y_test, test_data_prediction)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.4346280446064458


# Other models

In [66]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score

In [67]:
models = [
    LinearRegression(),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    SVR(),
    KNeighborsRegressor()
]

# Define a list of model names for reference
model_names = [
    'Linear Regression',
    'Decision Tree',
    'Random Forest',
    'Support Vector Regression',
    'K-Nearest Neighbors',
    
]

In [68]:
results_df = pd.DataFrame(columns=['Model', 'R Squared (Train)', 'R Squared (Test)', 'CV Score'])
results_df

,Model,R Squared (Train),R Squared (Test),CV Score


In [69]:
# Initialize an empty DataFrame to store results
results_data = []

for model, name in zip(models, model_names):
    # Fit the model
    model.fit(X_train, Y_train)
    
    # Prediction on training data
    train_predictions = model.predict(X_train)
    r2_train = r2_score(Y_train, train_predictions)
    
    # Prediction on test data
    test_predictions = model.predict(X_test)
    r2_test = r2_score(Y_test, test_predictions)
    
    # Mean Squared Error
    mse = mean_squared_error(Y_test, test_predictions)
    
    # Perform cross-validation
    cv_scores = cross_val_score(model, X, Y, scoring='r2', cv=5)
    cv_score = np.mean(np.abs(cv_scores))
    
    # Append results to list
    results_data.append({
        'Model': name,
        'R Squared (Train)': r2_train,
        'R Squared (Test)': r2_test,
        'CV Score': cv_score,
        'Mean Squared Error': mse
     })

# Convert list of dictionaries to DataFrame
results_df = pd.DataFrame(results_data)

In [70]:
# Print results
print(results_df)

                       Model  R Squared (Train)  R Squared (Test)  CV Score  \
0          Linear Regression           0.574493          0.591511  0.578012   
1              Decision Tree           1.000000          0.476560  0.430280   
2              Random Forest           0.958063          0.717730  0.699121   
3  Support Vector Regression           0.216820          0.209168  0.212628   
4        K-Nearest Neighbors           0.432791          0.165316  0.147828   

   Mean Squared Error  
0            0.434617  
1            0.556921  
2            0.300324  
3            0.841416  
4            0.888073  


# Classification

In [71]:
big_mart_data['Item_Fat_Content'].unique()

array([0, 1])

In [72]:
big_mart_data["Outlet_Size"].unique()

array([1, 2, 0])

In [73]:
big_mart_data['Outlet_Type'].unique()

array([1, 2, 0, 3])

In [74]:
big_mart_data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,156,9.30,0,0.015920,4,249.8092,9,1999,1,0,1,8.225808
1,8,5.92,1,0.019095,14,48.2692,3,2009,1,2,2,6.096776
2,662,17.50,0,0.016621,10,141.6180,9,1999,1,0,1,7.648868
3,1121,19.20,1,0.064037,6,182.0950,0,1998,2,2,0,6.597664
4,1297,8.93,0,0.064037,9,53.8614,1,1987,0,2,1,6.903451


In [75]:
# Create correlation matrix
corr_matrix = big_mart_data.corr()

# Create heatmap using Plotly
fig = go.Figure(data=go.Heatmap(
                   z=corr_matrix.values,
                   x=corr_matrix.columns,
                   y=corr_matrix.index,
                   colorscale='magma',
                   colorbar=dict(title='Correlation'),
                   zmax=0.9,  # Set max value for color scale
                   zmin=-0.9  # Set min value for color scale
                 ))

# Update layout
fig.update_layout(
    title='Correlation Heatmap',
    width=800,
    height=800,
    xaxis_title='Features',
    yaxis_title='Features',
    margin=dict(l=50, r=50, t=50, b=50)
)

fig.show()

# Feature selection

In [76]:
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
threshold = 0.5
highly_correlated = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]
selected_features = big_mart_data.drop(highly_correlated, axis=1)

print(selected_features.columns)

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type'],
      dtype='object')


In [77]:
import plotly.express as px

# Group by 'Item_Fat_Content' and calculate the size of each group
pie_data = big_mart_data.groupby('Item_Fat_Content').size().reset_index(name='count')

# Create a pie plot using Plotly Express
fig = px.pie(pie_data, values='count', names='Item_Fat_Content', 
             title='Item Fat Content Distribution', 
             labels={'Item_Fat_Content': 'Fat Content'})

# Show the plot
fig.show()


In [78]:
pie_data

,Item_Fat_Content,count
0,0,5517
1,1,3006


In [79]:

fig = px.histogram(big_mart_data['Item_Fat_Content'], x='Item_Fat_Content')
fig.update_layout(
    title='imBalanced Item Fat Content',
    xaxis_title='Item Fat Content',
    yaxis_title='Count',
    bargap=0.1,
    margin=dict(l=50, r=50, t=50, b=50),
    width=400,
    height=400
)
fig.update_xaxes(tickvals=[0, 1])
fig.show()

In [80]:
from imblearn.over_sampling import RandomOverSampler

In [81]:
X = big_mart_data.drop("Item_Fat_Content", axis = 1)
y = big_mart_data['Item_Fat_Content']

In [82]:
oversample = RandomOverSampler(sampling_strategy='minority')

X_resampled, y_resampled = oversample.fit_resample(X, y)

In [83]:
X_resampled.shape

(11034, 11)

In [84]:
new_df_balanced = pd.concat([X_resampled, y_resampled], axis = 1)

In [85]:
new_df_balanced.shape

(11034, 12)

In [86]:
# Create countplot using Plotly Express
fig = px.histogram(new_df_balanced, x='Item_Fat_Content')
# Update layout
fig.update_layout(
    title='Balanced Item Fat Content',
    xaxis_title='Item Fat Content',
    yaxis_title='Count',
    bargap=0.1,
    margin=dict(l=50, r=50, t=50, b=50),
    width=400,
    height=400
)
# Update x-axis tick values
fig.update_xaxes(tickvals=[0, 1])
fig.show()

In [87]:
X = new_df_balanced[['Item_Identifier', 'Item_Weight', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type']]
y = new_df_balanced['Item_Fat_Content']

# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

In [88]:
print("X_train",X_train.shape)
print("y_train",y_train.shape)
print("X_test",X_test.shape)
print("y_test",y_test.shape)

X_train (7723, 10)
y_train (7723,)
X_test (3311, 10)
y_test (3311,)


# RandomForestClassifier

In [89]:
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
model3 = RandomForestClassifier()
model3.fit(X_train, y_train)

RandomForestClassifier()

In [90]:
y_pred3 = model3.predict(X_test)
y_pred_prob = model3.predict_proba(X_test)[:, 1]

In [91]:
from sklearn.metrics import confusion_matrix
cm2 = confusion_matrix(y_test, y_pred3)
cm2

array([[1507,  125],
       [  39, 1640]])

In [92]:
fig = go.Figure(data=go.Heatmap(
                   z=cm2,
                   colorscale='Blues',  
                   hoverongaps=False
                 ))

fig.update_layout(
    title='Confusion Matrix',
    xaxis_title='Predicted',
    yaxis_title='Actual',
    width=400,
    height=300,
    margin=dict(l=50, r=50, t=50, b=50),
#     xaxis_showgrid=False  # Hide horizontal grid lines
)


fig.update_xaxes(tickvals=[0, 1])

# Update x-axis tick values
fig.update_yaxes(tickvals=[0, 1])

fig.show()

# ROC and AUC

In [93]:
from sklearn.metrics import roc_curve,auc
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr,tpr)
fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', line=dict(color='blue', width=2), name=f'ROC curve (AUC = {roc_auc:.2f})', fill='tozeroy'))
fig.add_shape(type='line', x0=0, y0=0, x1=1, y1=1, line=dict(color='gray', dash='dash'))
fig.update_layout(
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    title='Receiver Operating Characteristic (ROC) Curve',
    legend=dict(x=0.98, y=0.02, bgcolor='rgba(255, 255, 255, 0.5)'),
    margin=dict(l=50, r=50, t=50, b=50),
    width=800,
    height=600
)

fig.add_annotation(
    x=0.5,
    y=0.5,
    text=f'AUC = {roc_auc:.2f}',
    showarrow=False,
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="black"
    )
)

fig.show()

In [94]:
from sklearn.metrics import accuracy_score
print("Accuracy Score of RandomForestClassifier is ",accuracy_score(y_test, y_pred3))

Accuracy Score of RandomForestClassifier is  0.9504681365146481


In [95]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred3))

              precision    recall  f1-score   support

           0       0.97      0.92      0.95      1632
           1       0.93      0.98      0.95      1679

    accuracy                           0.95      3311
   macro avg       0.95      0.95      0.95      3311
weighted avg       0.95      0.95      0.95      3311



In [96]:
y_pred3=model3.predict(X)
# perform cross-validation
cv_score = cross_val_score(model3, X, y, scoring='accuracy', cv=5)
cv_score = np.abs(np.mean(cv_score))

print("Model Report")

print("CV Score:", cv_score)

Model Report
CV Score: 0.9552296513072844


# LogisticRegression 

In [97]:
from sklearn.linear_model import LogisticRegression
model3 = LogisticRegression()
model3.fit(X_train, y_train)

LogisticRegression()

In [98]:
y_pred3 = model3.predict(X_test)
y_pred_prob = model3.predict_proba(X_test)[:, 1]

In [99]:
cm2 = confusion_matrix(y_test, y_pred3)
cm2

array([[967, 665],
       [724, 955]])

In [100]:
fig = go.Figure(data=go.Heatmap(
                   z=cm2,
                   colorscale='Blues',  
                   hoverongaps=False
                 ))

fig.update_layout(
    title='Confusion Matrix',
    xaxis_title='Predicted',
    yaxis_title='Actual',
    width=400,
    height=300,
    margin=dict(l=50, r=50, t=50, b=50),
#     xaxis_showgrid=False  # Hide horizontal grid lines
)

fig.update_xaxes(tickvals=[0, 1])

fig.update_yaxes(tickvals=[0, 1])

fig.show()

In [101]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)
fig = go.Figure()


fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', line=dict(color='blue', width=2), name=f'ROC curve (AUC = {roc_auc:.2f})', fill='tozeroy'))


fig.add_shape(type='line', x0=0, y0=0, x1=1, y1=1, line=dict(color='gray', dash='dash'))


fig.update_layout(
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    title='Receiver Operating Characteristic (ROC) Curve',
    legend=dict(x=0.98, y=0.02, bgcolor='rgba(255, 255, 255, 0.5)'),
    margin=dict(l=50, r=50, t=50, b=50),
    width=800,
    height=600
)

fig.add_annotation(
    x=0.5,
    y=0.5,
    text=f'AUC = {roc_auc:.2f}',
    showarrow=False,
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="black"
    )
)

fig.show()


In [102]:
print("Accuracy Score of LogisticRegression is ",accuracy_score(y_test, y_pred3))

Accuracy Score of LogisticRegression is  0.5804892781636968


In [103]:
print(classification_report(y_test, y_pred3))

              precision    recall  f1-score   support

           0       0.57      0.59      0.58      1632
           1       0.59      0.57      0.58      1679

    accuracy                           0.58      3311
   macro avg       0.58      0.58      0.58      3311
weighted avg       0.58      0.58      0.58      3311



In [104]:
y_pred3=model3.predict(X)
# perform cross-validation
cv_score = cross_val_score(model3, X, y, scoring='accuracy', cv=5)
cv_score = np.abs(np.mean(cv_score))

print("Model Report")

print("CV Score:", cv_score)

Model Report
CV Score: 0.5771270099547265


# other classification models

In [105]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [106]:
models = [
    DecisionTreeClassifier(),
    GaussianNB(),
    SVC(probability=True),
    KNeighborsClassifier(),
    GradientBoostingClassifier()
]

# Define model names for reference
model_names = [
    'Decision Tree',
    'Support Vector Classifier',
    'Gaussian Naive Bayes',
    'K-Nearest Neighbors',
    'Gradient Boosting'
]

In [107]:
results = []

for model, name in zip(models, model_names):
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]

    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    acc_score = accuracy_score(y_test, y_pred)
    
    cls_report = classification_report(y_test, y_pred)
    
    cv_score = cross_val_score(model, X, y, scoring='accuracy', cv=5)
    cv_score = np.abs(np.mean(cv_score))

    results.append({
        'Model': name,
        'ROC_AUC': roc_auc,
        'Accuracy': acc_score,
        'Classification_Report': cls_report,
        'CV_Score': cv_score
    })

In [108]:
for result in results:
    print("Model:", result['Model'])
    print("ROC AUC:", result['ROC_AUC'])
    print("Accuracy:", result['Accuracy'])
    print("Classification Report:")
    print(result['Classification_Report'])
    print("CV Score:", result['CV_Score'])
    print()
    print()

Model: Decision Tree
ROC AUC: 0.9524398130306322
Accuracy: 0.9525823014195107
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      1632
           1       0.95      0.96      0.95      1679

    accuracy                           0.95      3311
   macro avg       0.95      0.95      0.95      3311
weighted avg       0.95      0.95      0.95      3311

CV Score: 0.9579493419314872


Model: Support Vector Classifier
ROC AUC: 0.691546161347207
Accuracy: 0.6460283902144367
Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.59      0.62      1632
           1       0.64      0.70      0.67      1679

    accuracy                           0.65      3311
   macro avg       0.65      0.65      0.64      3311
weighted avg       0.65      0.65      0.64      3311

CV Score: 0.6405656443829717


Model: Gaussian Naive Bayes
ROC AUC: 0.6965787729624309
Accuracy: 0.649

# Clustring

In [109]:
from sklearn.cluster import KMeans
X = X[['Item_Visibility', 'Item_MRP']].values 
k = 10
kmeans = KMeans(n_clusters=k)
kmeans.fit(X)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
print("Labels: ",labels)
print("centroids: ",centroids)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Labels:  [5 9 6 ... 1 5 5]
centroids:  [[6.74581901e-02 1.71365966e+02]
 [6.57256767e-02 8.54955879e+01]
 [6.93751265e-02 2.22836657e+02]
 [6.93434245e-02 5.75126275e+01]
 [6.31731375e-02 1.22886573e+02]
 [6.73763520e-02 2.53264161e+02]
 [6.83668693e-02 1.49899963e+02]
 [6.70203358e-02 1.90299774e+02]
 [6.70751291e-02 1.04653686e+02]
 [7.23607939e-02 4.13913912e+01]]


In [110]:
# finding The Within-Cluster Sum of Squares value for different number of clusters

wcss = []

for i in range(1,11):
  kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
  kmeans.fit(X)

  wcss.append(kmeans.inertia_)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will chang

In [111]:
x_values = list(range(1, 11))

trace = go.Scatter(
    x=x_values,  
    y=wcss,  
    mode='lines+markers',
    marker=dict(color='blue'),  # Marker color
    name='WCSS'  
)
layout = go.Layout(
    title='Elbow Method for Optimal Number of Clusters',
    xaxis=dict(title='Number of Clusters'),  
    yaxis=dict(title='WCSS'),  
)
fig = go.Figure(data=[trace], layout=layout)
fig.show()


In [112]:
kmeans = KMeans(n_clusters=4, init='k-means++', random_state=0)
Y = kmeans.fit_predict(X)
X



/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



array([[1.59199042e-02, 2.49809200e+02],
       [1.90947454e-02, 4.82692000e+01],
       [1.66211748e-02, 1.41618000e+02],
       ...,
       [1.04554090e-01, 8.18592000e+01],
       [3.61131020e-02, 2.50309200e+02],
       [1.30608816e-01, 2.66888400e+02]])

In [113]:
# Define colors for clusters
colors = ['green', 'red', 'yellow', 'violet']
fig = go.Figure()
for i in range(len(colors)):
    fig.add_trace(go.Scatter(
        x=X[Y == i, 0],  # X coordinates for points in cluster i
        y=X[Y == i, 1],  # Y coordinates for points in cluster i
        mode='markers',
        marker=dict(color=colors[i], size=10),
        name=f'Cluster {i + 1}'
    ))

# Add centroids as scatter plot
fig.add_trace(go.Scatter(
    x=kmeans.cluster_centers_[:, 0],  # X coordinates for centroids
    y=kmeans.cluster_centers_[:, 1],  # Y coordinates for centroids
    mode='markers',
    marker=dict(color='cyan', size=12, symbol='x'),
    name='Centroids'
))

# Update layout
fig.update_layout(
    title='Clusters and Centroids',
    xaxis_title='Item Visibility',
    yaxis_title='Item MRP',
    showlegend=True
)

# Show plot
fig.show()

# Detect and Removing outliers

In [114]:
import plotly.subplots as sp

def detect_outliers(df, feature):
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1

    upper_limit = Q3 + 1.5 * IQR
    lower_limit = Q1 - 1.5 * IQR
    return upper_limit, lower_limit

upper, lower = detect_outliers(big_mart, "Item_Visibility")
print("Upper limit: ", upper)
print("Lower limit: ", lower)

fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Item_Visibility Distribution before Removing Outliers", "Item_Visibility Distribution after Removing Outliers"))

# Create box plot before removing outliers
boxplot_before = go.Box(y=big_mart['Item_Visibility'], name="Before", boxpoints='all', jitter=0.3, pointpos=-1.8)
fig.add_trace(boxplot_before, row=1, col=1)

# Remove outliers
big_mart_filtered = big_mart[(big_mart['Item_Visibility'] > lower) & (big_mart['Item_Visibility'] < upper)]

# Create box plot after removing outliers
boxplot_after = go.Box(y=big_mart_filtered['Item_Visibility'], name="After", boxpoints='all', jitter=0.3, pointpos=-1.8)
fig.add_trace(boxplot_after, row=1, col=2)

fig.update_layout(title_text='Item_Visibility Distribution before VS after removing outliers')
fig.update_xaxes(title_text="Item Visibility", row=1, col=1)
fig.update_xaxes(title_text="Item Visibility", row=1, col=2)
fig.update_yaxes(title_text="Distribution", row=1, col=1)
fig.update_yaxes(title_text="Distribution", row=1, col=2)

fig.show()


Upper limit:  0.17711420193117242
Lower limit:  -0.05418883499091427


In [115]:
upper, lower = detect_outliers(big_mart, "Item_Outlet_Sales")
print("Upper limit: ", upper)
print("Lower limit: ", lower)

fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Item_Outlet_Sales Distribution before Removing Outliers", "Item_Outlet_Sales Distribution after Removing Outliers"))

# Create box plot before removing outliers
boxplot_before = go.Box(y=big_mart['Item_Visibility'], name="Before", boxpoints='all', jitter=0.3, pointpos=-1.8)
fig.add_trace(boxplot_before, row=1, col=1)

# Remove outliers
big_mart_filtered = big_mart[(big_mart['Item_Outlet_Sales'] > lower) & (big_mart['Item_Outlet_Sales'] < upper)]

# Create box plot after removing outliers
boxplot_after = go.Box(y=big_mart_filtered['Item_Outlet_Sales'], name="After", boxpoints='all', jitter=0.3, pointpos=-1.8)
fig.add_trace(boxplot_after, row=1, col=2)

fig.update_layout(title_text='Item_Outlet_Sales Distribution before VS after removing outliers')
fig.update_xaxes(title_text="Item Outlet Sales", row=1, col=1)
fig.update_xaxes(title_text="Item Outlet Sales", row=1, col=2)
fig.update_yaxes(title_text="Distribution", row=1, col=1)
fig.update_yaxes(title_text="Distribution", row=1, col=2)

fig.show()


Upper limit:  10.008153250046428
Lower limit:  4.759472291066638


# Dimentionality reduction

In [116]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
numeric_df = big_mart_data.drop(['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type'], axis=1)
numeric_df.fillna(numeric_df.mean(), inplace=True)

scaler = StandardScaler()
scaled_df = scaler.fit_transform(numeric_df)

pca = PCA(n_components=2)  
pca.fit(scaled_df)
pca_df = pca.transform(scaled_df)

pca_columns = ['Principal Component 1', 'Principal Component 2']  # Change column names as needed
pca_df = pd.DataFrame(data=pca_df, columns=pca_columns)

final_df = pd.concat([pca_df, big_mart_data[['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']]], axis=1)

print(final_df)


      Principal Component 1  Principal Component 2  Item_Identifier  \
0                 -2.058970              -0.200882              156   
1                  1.380370              -2.282065                8   
2                 -0.627525              -0.720943              662   
3                 -0.028628               0.233225             1121   
4                  1.374907               0.276940             1297   
...                     ...                    ...              ...   
8518              -1.023704               1.090699              370   
8519               0.869951              -0.940616              897   
8520               0.446228              -1.324824             1357   
8521               0.536260              -0.225056              681   
8522               0.980167              -0.636986               50   

      Item_Fat_Content  Item_Type  Outlet_Identifier  Outlet_Size  \
0                    0          4                  9            1   
1        

In [117]:
fig = px.scatter(final_df, x='Principal Component 1', y='Principal Component 2', 
                 color='Outlet_Type', hover_data=['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type'],
                 title='PCA Visualization of Big Mart Data')
fig.show()
